In [2]:
import pandas as pd
import numpy as np

In [8]:
working_data=pd.read_csv('augmented_data.csv')
working_data.head(3)

,updated_label,source_article
0,faulty generalization,"""Annie must like Starbucks because all white g..."
1,faulty generalization,It is warmer this year in Las Vegas as compare...
2,faulty generalization,"""The two courses I took at UF were not very in..."


In [9]:
working_data.shape

(14000, 2)

In [10]:
working_data=working_data.drop_duplicates()
working_data.shape

(13969, 2)

In [12]:
working_data.tail(3)

,updated_label,source_article
13997,miscellaneous,Who could disagree over the bipartisan plan to...
13998,miscellaneous,Who could disagree on the bipartisan plan to r...
13999,miscellaneous,Who could disagree with the bipartisan plan an...


In [15]:
miscellaneous_count = working_data[working_data['updated_label'] == 'miscellaneous'].shape[0]

print(f"Number of rows with 'miscellaneous' label: {miscellaneous_count}")

Number of rows with 'miscellaneous' label: 999


In [16]:
working_data=working_data[working_data['updated_label']!='miscellaneous']
working_data.shape

(12970, 2)

In [17]:
class_counts = working_data['updated_label'].value_counts()

print(class_counts)

false causality           999
fallacy of logic          999
fallacy of relevance      999
fallacy of credibility    999
faulty generalization     998
appeal to emotion         998
fallacy of extension      998
circular reasoning        997
ad hominem                997
equivocation              997
intentional               997
ad populum                996
false dilemma             996
Name: updated_label, dtype: int64


In [18]:
working_data.rename(columns={'updated_label': 'label'}, inplace=True)


C:\Users\User\AppData\Local\Temp\ipykernel_29856\2329233188.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_data.rename(columns={'updated_label': 'label'}, inplace=True)


In [19]:
working_data.head()

,label,source_article
0,faulty generalization,"""Annie must like Starbucks because all white g..."
1,faulty generalization,It is warmer this year in Las Vegas as compare...
2,faulty generalization,"""The two courses I took at UF were not very in..."
3,faulty generalization,A driver with a New York license plate cuts yo...
4,faulty generalization,"""Four out of five dentists recommend Happy Glo..."


In [21]:
import argparse
from torch import nn
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np
import os
import json
from sklearn.metrics import classification_report, confusion_matrix
import glob, shutil


In [22]:
working_data.to_csv('readyData.csv', index=False)

In [23]:
mappings = {
    'faulty generalization': 'component fallacy', 
    'false causality': 'component fallacy', 
    'false dilemma': 'component fallacy', 
    'circular reasoning': 'component fallacy', 
    'fallacy of logic': 'component fallacy', 
    'fallacy of extension': 'component fallacy', 
    'fallacy of relevance': 'fallacy of relevance', 
    'ad populum': 'fallacy of relevance', 
    'ad hominem': 'fallacy of relevance', 
    'fallacy of credibility': 'fallacy of relevance', 
    'intentional': 'fallacy of relevance', 
    'equivocation': 'equivocation', 
    'appeal to emotion': 'fallacy of relevance'
   
}

labels = {
    'fallacy of relevance': 0,
    'component fallacy': 1, 
    'equivocation': 2
}

In [32]:
working_data.rename(columns={'source_article': 'text'}, inplace=True)


C:\Users\User\AppData\Local\Temp\ipykernel_29856\3348623768.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_data.rename(columns={'source_article': 'text'}, inplace=True)


In [33]:
broad_classification = []
mapped_labels = []

# Iterate through the working_data's labels and apply the mappings
for label in working_data['label']:
    broad_classification.append(mappings[label])
    mapped_labels.append(labels[mappings[label]])

# Add the new mappings as columns to the working_data
working_data['broad_classification'] = broad_classification
working_data['mapped_labels'] = mapped_labels

# Display the updated working_data
print(working_data)

                       label  \
0      faulty generalization   
1      faulty generalization   
2      faulty generalization   
3      faulty generalization   
4      faulty generalization   
...                      ...   
12998           equivocation   
12999           equivocation   
13000           equivocation   
13001           equivocation   
13002           equivocation   

                                                    text broad_classification  \
0      "Annie must like Starbucks because all white g...    component fallacy   
1      It is warmer this year in Las Vegas as compare...    component fallacy   
2      "The two courses I took at UF were not very in...    component fallacy   
3      A driver with a New York license plate cuts yo...    component fallacy   
4      "Four out of five dentists recommend Happy Glo...    component fallacy   
...                                                  ...                  ...   
12998  Board Member: “If this corporation is goi

C:\Users\User\AppData\Local\Temp\ipykernel_29856\2136128952.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_data['broad_classification'] = broad_classification
C:\Users\User\AppData\Local\Temp\ipykernel_29856\2136128952.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_data['mapped_labels'] = mapped_labels


In [34]:
working_data.head(7)

,label,text,broad_classification,mapped_labels
0,faulty generalization,"""Annie must like Starbucks because all white g...",component fallacy,1
1,faulty generalization,It is warmer this year in Las Vegas as compare...,component fallacy,1
2,faulty generalization,"""The two courses I took at UF were not very in...",component fallacy,1
3,faulty generalization,A driver with a New York license plate cuts yo...,component fallacy,1
4,faulty generalization,"""Four out of five dentists recommend Happy Glo...",component fallacy,1
5,faulty generalization,No one in your family surfs? But I thought you...,component fallacy,1
6,faulty generalization,All women are bad drivers.,component fallacy,1


In [36]:
working_data = working_data.drop(['label', 'mapped_labels'], axis=1)

In [39]:
working_data.rename(columns={'broad_classification': 'label'}, inplace=True)


In [40]:
working_data.head()

,text,label
0,"""Annie must like Starbucks because all white g...",component fallacy
1,It is warmer this year in Las Vegas as compare...,component fallacy
2,"""The two courses I took at UF were not very in...",component fallacy
3,A driver with a New York license plate cuts yo...,component fallacy
4,"""Four out of five dentists recommend Happy Glo...",component fallacy


In [55]:
working_data.to_csv('CoarseClassData/trainingset.csv', index=False)

In [41]:
from sklearn.model_selection import train_test_split


In [42]:
train_dataset, temp_dataset = train_test_split(working_data, test_size=0.4, random_state=42)  # 60% train, 40% temp
test_dataset, val_dataset = train_test_split(temp_dataset, test_size=0.5, random_state=42)  # Split temp into 50% test, 50% val

# Save the datasets to CSV files
train_dataset.to_csv('CoarseClassData/train_dataset.csv', index=False)
test_dataset.to_csv('CoarseClassData/test_dataset.csv', index=False)
val_dataset.to_csv('CoarseClassData/val_dataset.csv', index=False)

print("Datasets have been successfully split and saved.")

Datasets have been successfully split and saved.


In [29]:
readyData=pd.read_csv('readyData.csv')

In [30]:
train_dataset, temp_dataset = train_test_split(readyData, test_size=0.4, random_state=42)  # 60% train, 40% temp
test_dataset, val_dataset = train_test_split(temp_dataset, test_size=0.5, random_state=42)  # Split temp into 50% test, 50% val

# Save the datasets to CSV files
train_dataset.to_csv('FineClassData/train_dataset.csv', index=False)
test_dataset.to_csv('FineClassData/test_dataset.csv', index=False)
val_dataset.to_csv('FineClassData/val_dataset.csv', index=False)

print("Datasets have been successfully split and saved.")

Datasets have been successfully split and saved.


In [31]:
readyData.head()

,label,text
0,faulty generalization,"""Annie must like Starbucks because all white g..."
1,faulty generalization,It is warmer this year in Las Vegas as compare...
2,faulty generalization,"""The two courses I took at UF were not very in..."
3,faulty generalization,A driver with a New York license plate cuts yo...
4,faulty generalization,"""Four out of five dentists recommend Happy Glo..."


In [46]:
readyData['label'] = readyData['label'].apply(lambda x: x.replace('“', '"').replace('”', '"'))


In [51]:
readyData['text'] = readyData['text'].apply(lambda x: x.replace('“', '"').replace('”', '"'))
readyData['text'] = readyData['text'].apply(lambda x: x.replace('“', ' ').replace('”', '"').replace('\n', ' '))
readyData=readyData[['text','label']]

In [52]:
train_dataset, temp_dataset = train_test_split(readyData, test_size=0.4, random_state=42)  # 60% train, 40% temp
test_dataset, val_dataset = train_test_split(temp_dataset, test_size=0.5, random_state=42)  # Split temp into 50% test, 50% val

# Save the datasets to CSV files
train_dataset.to_csv('FineClassData/train_dataset.csv', index=False)
test_dataset.to_csv('FineClassData/test_dataset.csv', index=False)
val_dataset.to_csv('FineClassData/val_dataset.csv', index=False)

print("Datasets have been successfully split and saved.")

Datasets have been successfully split and saved.


In [44]:
unique_values = readyData['label'].unique()


In [45]:
print(unique_values)

['faulty generalization' 'false causality' 'circular reasoning'
 'ad populum' 'ad hominem' 'fallacy of logic' 'appeal to emotion'
 'false dilemma' 'equivocation' 'fallacy of extension'
 'fallacy of relevance' 'fallacy of credibility' 'intentional']


In [3]:
data=pd.read_csv("data.csv")
data.shape

(2452, 6)